In [1]:
# convolution
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
def corr2d(X, K):
    """2-dimensional cross-correlation (convolution)"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j: j + w] * K).sum()
    return Y

In [3]:
X = torch.tensor([[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]])
K = torch.tensor([[0.0,1.0],[2.0,3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [4]:
# convolution layer
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [5]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [6]:
k = torch.tensor([[1.0,-1.0]])  # kernel 1 * 2
Y = corr2d(X, k)
Y  # 1 represents the edge from 1 to 0, -1 represents the edge from 0 to 1

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [7]:
corr2d(X.t(), k)  # edges disappear, so the k can only detect the vertical edges

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [8]:
# to make the conv to learn, so that can automatically generate complicated convolutions
# construct a 2-dimensional conv layer, which has one output channel and kernel in the shape of (1, 2)
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

# input and output with 4-dimensional format (batch_size, channel, h, w)
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # iterate kernel
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 2) % 2 == 0:
        print(f'epoch {i + 1}, loss {l.sum():.3f}')

conv2d.weight.data.reshape((1,2))

epoch 1, loss 32.329
epoch 3, loss 9.263
epoch 5, loss 3.127
epoch 7, loss 1.169
epoch 9, loss 0.460


tensor([[ 1.0417, -0.9313]])